# Chunking & Embeddings

In this notebook, we will create chunking strategies and use two different embedding models to vectorize the chunks. We will use the following chunking strategies:
- 1000 chunk size with 100 overlap
- 500 chunk size with 50 overlap
- 2500 chunk size with 300 overlap

We will use the following models:
- `text-embedding-ada-002`
- `text-embedding-3-large`

These created embeddings will be saved as a pickle file for further use.

## Imports and Setup

In [82]:
import pandas as pd
import time
import os

from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from openai import AzureOpenAI
import openai

import credentials

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import math

In [83]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

## Data Preparation and Chunking

Here we load the preprocessed data and chunk the texts with the defined parameters.

In [84]:
data = pd.read_csv('../data/processed/cleantech_processed.csv')
data.head()

print("Number of articles in the dataset: ", len(data))

,title,date,content,domain,url
0,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,Qatar Petroleum (QP) is targeting aggressive c...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
1,India Launches Its First 700 MW PHWR,2021-01-15,Nuclear Power Corp of India Ltd (NPCIL) synchr...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
2,New Chapter for US-China Energy Trade,2021-01-20,New US President Joe Biden took office this we...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
3,Japan: Slow Restarts Cast Doubt on 2030 Energy...,2021-01-22,The slow pace of Japanese reactor restarts con...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
4,NYC Pension Funds to Divest Fossil Fuel Shares,2021-01-25,Two of New York City's largest pension funds s...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...


In [86]:
CHUNK_SIZE = 2500
CHUNK_OVERLAP = 300
EMBEDDING_MODEL = "text-embedding-3-large"

In [87]:
def chunk_text(dataframe, text_column, chunk_size=1000, chunk_overlap=100):
    # Initialize RecursiveCharacterTextSplitter with dynamic parameters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    # Split text in the specified column
    dataframe['content_chunks'] = dataframe[text_column].apply(lambda text: text_splitter.split_text(text))
    
    # Flatten the DataFrame for individual chunk rows
    chunked_df = dataframe.explode('content_chunks').reset_index(drop=True)

    return chunked_df

chunked_data = chunk_text(data, text_column='content', chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
chunked_data.head()

,title,date,content,domain,url,content_chunks
0,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,Qatar Petroleum (QP) is targeting aggressive c...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,Qatar Petroleum (QP) is targeting aggressive c...
1,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,Qatar Petroleum (QP) is targeting aggressive c...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"in the Phase 1 trains. Exxon Mobil, Royal Dutc..."
2,India Launches Its First 700 MW PHWR,2021-01-15,Nuclear Power Corp of India Ltd (NPCIL) synchr...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,Nuclear Power Corp of India Ltd (NPCIL) synchr...
3,India Launches Its First 700 MW PHWR,2021-01-15,Nuclear Power Corp of India Ltd (NPCIL) synchr...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,dropped out of the project and another two off...
4,New Chapter for US-China Energy Trade,2021-01-20,New US President Joe Biden took office this we...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,New US President Joe Biden took office this we...


In [88]:
len(chunked_data)

21632

Now that we have chunked the texts into smaller segments, the next step is to pass these chunks through an embedding model to obtain their vector representations. The embedding model maps the textual information into high-dimensional vector spaces, where semantic similarities and relationships are preserved.

## Embedding Creation

This code below generates embeddings for the text data in batches while handling rate-limiting errors. It splits the input data into manageable batches, each of size `BATCH_SIZE` and processes them concurrently using `ThreadPoolExecutor` with up to `MAX_WORKERS` threads. The `embed_batch` function sends each batch to the embedding model, implements retry logic to handle rate-limiting errors by waiting and retrying up to three times and collects the embeddings. The `embed_in_batches` function coordinates this process, maps batches to threads, and aggregates the results into a single list, ensuring each text chunk receives its embedding. The entire pipeline is designed for scalability and robustness against API limits.

In [89]:
BATCH_SIZE = 50
MAX_WORKERS = 8  # Number of threads
RETRY_DELAY = 60  # Retry delay in seconds for rate-limiting errors

# Function to generate embeddings for a batch of texts with retry logic
def embed_batch(text_batch, model, retries=3):
    attempts = 0
    while attempts < retries:
        try:
            response = client.embeddings.create(input=text_batch, model=model)
            return [item.embedding for item in response.data]
        except Exception as e:
            if "rate limit" in str(e).lower() or "429" in str(e):
                print(f"Rate limit exceeded. Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)  # Wait before retrying
                attempts += 1
            else:
                print(f"Error embedding batch: {e}")
                return [None] * len(text_batch)
    print("Failed to embed after multiple attempts.")
    return [None] * len(text_batch)

# Function to process batches in parallel with rate limiting
def embed_in_batches(data, model=EMBEDDING_MODEL, batch_size=BATCH_SIZE, max_workers=MAX_WORKERS):
    total_chunks = len(data)
    num_batches = math.ceil(total_chunks / batch_size)
    
    # Create batches
    batches = [data[i * batch_size: (i + 1) * batch_size] for i in range(num_batches)]
    embeddings = [None] * total_chunks  # Placeholder for embeddings

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {}
        for i, batch in enumerate(batches):
            futures[executor.submit(embed_batch, batch, model)] = i
        
        # Process results as they complete
        for future in tqdm(as_completed(futures), total=len(futures), desc="Embedding in Parallel"):
            batch_index = futures[future]
            try:
                batch_embeddings = future.result()
                embeddings[batch_index * batch_size: (batch_index + 1) * batch_size] = batch_embeddings
            except Exception as e:
                print(f"Error processing batch {batch_index}: {e}")
    
    return embeddings

# Prepare data for embedding generation
texts_to_embed = chunked_data["content_chunks"].tolist()

# Run batch embedding generation with rate limiting
chunked_data['embeddings'] = embed_in_batches(
    texts_to_embed, 
    model=EMBEDDING_MODEL,
    batch_size=BATCH_SIZE,
    max_workers=MAX_WORKERS
)

Embedding in Parallel:   0%|          | 0/433 [00:00<?, ?it/s]

Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...


Embedding in Parallel:   2%|▏         | 9/433 [03:45<2:29:30, 21.16s/it]  

Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...


Embedding in Parallel:   3%|▎         | 14/433 [06:03<2:28:39, 21.29s/it]

...

Embedding in Parallel:  90%|████████▉ | 388/433 [2:02:16<08:59, 11.98s/it]

Rate limit exceeded. Retrying in 60 seconds...


Embedding in Parallel:  90%|████████▉ | 389/433 [2:03:01<16:01, 21.86s/it]

Rate limit exceeded. Retrying in 60 seconds...


Embedding in Parallel: 100%|██████████| 433/433 [2:11:11<00:00, 18.18s/it]


In [90]:
print("Dimension of the embeddings: ", len(chunked_data['embeddings'][0]))

Dimension of the embeddings:  3072


The `chunked_data` DataFrame is structured so that each row represents an individual **chunk** of text derived from the original documents, with a corresponding **embedding vector** stored in the `embeddings` column. Each embedding vector is a dense list of floating-point numbers (1536 dimensions with the `text-embedding-ada-002` model and 3072 dimensions for the `text-embedding-3-large` model) that encapsulates the semantic meaning of the text chunk.

These embeddings are critical for the RAG system, as they allow efficient similarity searches and retrieval tasks by representing the content of each text chunk in a high-dimensional vector space. When a query is embedded, the RAG system can quickly locate the most relevant text chunks by identifying embedding vectors in the database that are closest in meaning. This approach enables the system to retrieve contextually relevant information by comparing semantic relationships, streamlining the entire retrieval process.

In [91]:
import pickle

# Construct the filename dynamically
filename = f"../embeddings/recursive_{CHUNK_SIZE}_chunksize_{CHUNK_OVERLAP}_overlap_{EMBEDDING_MODEL.replace('-', '_')}.pkl"

# Save embeddings and corresponding chunked data to a file
with open(filename, 'wb') as f:
    pickle.dump(chunked_data, f)

print(f"Embeddings and chunked data saved to {filename}")

Embeddings and chunked data saved to ../data/experiments/recursive_2500_chunksize_300_overlap_text_embedding_3_large.pkl


metric assessment of embedding results on artificially created chunk-query pairs (intrinsic evaluation)